In [33]:
import openbabel
import multiprocessing as mp
import nglview as nv
import pandas as pd
import pybel

from Bio.PDB    import PDBParser
from meeko      import MoleculePreparation, PDBQTWriterLegacy
from rdkit      import Chem
from rdkit.Chem import AllChem
from vina       import Vina
from pdbfixer   import PDBFixer
from openmm.app import PDBFile

In [2]:
mp.cpu_count()

20

# Chemdbl

In [3]:
chembl = pd.read_table('data/chembl/chembl_33_chemreps.txt', sep='\t')

In [7]:
smiles = chembl.canonical_smiles.tolist()

# Processing

In [8]:
def smiles_to_rdkit_molecule(smiles_string):
    
    # Convert the SMILES string to an RDKit molecule object
    smiles_molecule = Chem.MolFromSmiles(smiles_string)

    # Add hydrogen atoms to the molecule
    molecule_with_hydrogens = Chem.AddHs(smiles_molecule)
    
    print(type(molecule_with_hydrogens))
    
    # Generate 3D coordinates
    AllChem.EmbedMolecule(molecule_with_hydrogens, AllChem.ETKDG())

    # Generate 3D coordinates
    if AllChem.EmbedMolecule(molecule_with_hydrogens, AllChem.ETKDG()) >= 0:
        # Optionally, optimize the geometry only if EmbedMolecule is successful
        AllChem.UFFOptimizeMolecule(molecule_with_hydrogens)
        success = 'Successfull'
    else:
        success = 'Not succesfull'
    
    return (molecule_with_hydrogens, success)

In [9]:
def parallel_smiles_to_rdkit_molecule(list_smiles_string, num_cores=mp.cpu_count()):
    
    with mp.Pool(num_cores) as pool:
        rdkit_objects, success  = pool.map(smiles_to_rdkit_molecule, list_smiles_string)
        
    return rdkit_objects, success

# Get molfile and save it to .pdb

In [10]:
%%time

molfile, success = smiles_to_rdkit_molecule(smiles[0])

<class 'rdkit.Chem.rdchem.Mol'>
CPU times: user 84.5 ms, sys: 1.82 ms, total: 86.3 ms
Wall time: 86 ms


In [11]:
Chem.MolToPDBFile(molfile, "data/molecule.pdb")

# Fixing the protein

In [ ]:
ph = 7.4  

In [15]:
pdb_filename = 'data/angptl7.pdb'  
fixer        = PDBFixer(filename=pdb_filename)

fixer.findMissingResidues()
fixer.findMissingAtoms()
fixer.addMissingAtoms()
fixer.addMissingHydrogens(ph)

PDBFile.writeFile(fixer.topology, fixer.positions, open('data/angptl7_fixed.pdbqt'  , 'w'))

# Fixing the molecule 

In [16]:
pdb_filename = 'data/molecule.pdb'
fixer        = PDBFixer(filename=pdb_filename)

fixer.findMissingResidues()
fixer.findMissingAtoms()
fixer.addMissingAtoms()
fixer.addMissingHydrogens(ph)

PDBFile.writeFile(fixer.topology, fixer.positions, open('data/molecule_fixed.pdbqt', 'w'))

# Preparation

In [19]:
preparator = MoleculePreparation()
mol_setups = preparator.prepare(molfile)
for setup in mol_setups:
    pdbqt_string, is_ok, error_msg = PDBQTWriterLegacy.write_string(setup)
    if is_ok:
        print(pdbqt_string, end="")

REMARK SMILES Cc1cc(-c2csc(N=C(N)N)n2)cn1C
REMARK SMILES IDX 5 1 6 2 7 3 8 4 13 5 4 6 1 7 2 8 3 9 15 10 14 11 16 12 9 13
REMARK SMILES IDX 10 14 11 15 12 16
REMARK H PARENT 11 17 11 18 12 19 12 20
REMARK Flexibility Score: inf
ROOT
ATOM      1  C   UNL     1       0.339   0.376  -0.475  1.00  0.00     0.085 A 
ATOM      2  C   UNL     1       0.883   1.352  -1.312  1.00  0.00     0.097 A 
ATOM      3  S   UNL     1       2.542   1.095  -1.399  1.00  0.00    -0.103 S 
ATOM      4  C   UNL     1       2.504  -0.215  -0.339  1.00  0.00     0.212 A 
ATOM      5  N   UNL     1       1.242  -0.500   0.044  1.00  0.00    -0.218 NA
ENDROOT
BRANCH   1   6
ATOM      6  C   UNL     1      -1.099   0.267  -0.176  1.00  0.00     0.019 A 
ATOM      7  C   UNL     1      -4.648   1.172  -0.317  1.00  0.00     0.064 C 
ATOM      8  C   UNL     1      -3.283   0.603  -0.086  1.00  0.00     0.015 A 
ATOM      9  C   UNL     1      -2.114   1.088  -0.642  1.00  0.00     0.033 A 
ATOM     10  N   UNL     

# Calculate geometric center

In [24]:
def calculate_geometric_center(pdb_filename):
    parser = PDBParser()
    structure = parser.get_structure('structure', pdb_filename)
    atom_coords = [atom.get_coord() for atom in structure.get_atoms()]
    x_coords = [coord[0] for coord in atom_coords]
    y_coords = [coord[1] for coord in atom_coords]
    z_coords = [coord[2] for coord in atom_coords]
    center_x = sum(x_coords) / len(x_coords)
    center_y = sum(y_coords) / len(y_coords)
    center_z = sum(z_coords) / len(z_coords)
    return [center_x, center_y, center_z]

In [82]:
pdb_filename = 'data/angptl7_fixed.pdbqt'
center = calculate_geometric_center(pdb_filename)
print('Geometric center:', center)

Geometric center: (10.270508859536989, 2.4234355613117953, -3.625699998706996)


/home/users/tlmenest/anaconda3/envs/chem/lib/python3.10/site-packages/Bio/PDB/PDBParser.py:388: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 5543
  warnings.warn(


# Docking

In [23]:
v = Vina(sf_name='vina')

v.set_receptor('data/angptl7_fixed.pdbqt')
v.set_ligand_from_string(pdbqt_string)

In [25]:
%%time

v.compute_vina_maps(center=calculate_geometric_center('data/angptl7_fixed.pdbqt'), box_size=[100, 100, 100])

# Score the current pose
energy = v.score()
print('Score before minimization: %.3f (kcal/mol)' % energy[0])

# Minimized locally the current pose
energy_minimized = v.optimize()
print('Score after minimization : %.3f (kcal/mol)' % energy_minimized[0])
v.write_pose('1iep_ligand_minimized.pdbqt', overwrite=True)

# Dock the ligand
v.dock(exhaustiveness=32, n_poses=20)
v.write_poses('1iep_ligand_vina_out.pdbqt', n_poses=5, overwrite=True)

Computing Vina grid ... done.
Score before minimization: 230.296 (kcal/mol)
Score after minimization : 0.000 (kcal/mol)
Performing local search ... done.
Performing docking (random seed: 1591798047) ... 
0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
***************************************************
CPU times: user 1min 28s, sys: 3.39 s, total: 1min 31s
Wall time: 40.6 s


# Parse output

In [26]:
def parse_vina_output(vina_output_file):

    with open(vina_output_file, 'r') as file:
        lines = file.readlines()
    poses = []
    current_pose = []
    binding_affinity = None
    for line in lines:
        if line.startswith('REMARK VINA RESULT:'):
            # Extract binding affinity
            parts = line.split()
            binding_affinity = float(parts[3])  # Binding affinity is the 4th item
        elif line.startswith('MODEL'):
            current_pose = []
        elif line.startswith('ENDMDL'):
            poses.append((binding_affinity, current_pose))
        else:
            current_pose.append(line)
    return poses

In [27]:
vina_output_file = '1iep_ligand_vina_out.pdbqt'
poses = parse_vina_output(vina_output_file)

for i, (binding_affinity, pose) in enumerate(poses):
    print(f"Pose {i + 1}: Binding Affinity = {binding_affinity} kcal/mol, Number of Atoms = {len(pose)}")

Pose 1: Binding Affinity = -5.496 kcal/mol, Number of Atoms = 36
Pose 2: Binding Affinity = -5.297 kcal/mol, Number of Atoms = 36
Pose 3: Binding Affinity = -5.105 kcal/mol, Number of Atoms = 36
Pose 4: Binding Affinity = -5.083 kcal/mol, Number of Atoms = 36
Pose 5: Binding Affinity = -5.053 kcal/mol, Number of Atoms = 36


# Visualize output

In [32]:
def convert_pdbqt_to_pdb(pdbqt_file, pdb_file):
    # Load the molecule from the PDBQT file
    for mol in openbabel.readfile('pdbqt', pdbqt_file):
        # Write the molecule to a PDB file
        mol.write(format='pdb', filename=pdb_file, overwrite=True)

#convert_pdbqt_to_pdb('1iep_ligand_vina_out.pdbqt', '1iep_ligand_vina_out.pdb')